<a href="https://colab.research.google.com/github/paveethranswam/Speech-style-transfer-and-ASR/blob/main/Wav2Vec2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers  datasets librosa jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.2 MB/s 
     |████████████████████████████████| 451 kB 61.5 MB/s 
     |████████████████████████████████| 7.6 MB 12.1 MB/s 
     |████████████████████████████████| 182 kB 18.4 MB/s 
     |████████████████████████████████| 212 kB 23.9 MB/s 
     |████████████████████████████████| 132 kB 66.8 MB/s 
     |████████████████████████████████| 127 kB 32.5 MB/s 
     |████████████████████████████████| 1.4 MB 51.4 MB/s 
     |████████████████████████████████| 2.2 MB 13.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
!apt-get install libsox-fmt-all libsox-dev sox > /dev/null
!python -m pip install torchaudio > /dev/null
!python -m pip install git+https://github.com/facebookresearch/WavAugment.git > /dev/null

  Running command git clone -q https://github.com/facebookresearch/WavAugment.git /tmp/pip-req-build-vu3hzehw


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from datasets import load_dataset, load_metric,Audio
import transformers
import random
import numpy as np
import pandas as pd
import re
from IPython.display import display, HTML
from transformers import AutoProcessor
import librosa
import torch
from datasets import ClassLabel
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR
from transformers import get_scheduler
import augment
import json
from jiwer import wer
import os
import tqdm
from functools import partial
from IPython.display import display, HTML

In [5]:
# Get device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda:0 device


In [6]:
minds = load_dataset("PolyAI/minds14", name="en-US", split="train[:100]")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset minds14 downloaded and prepared to /root/.cache/huggingface/datasets/PolyAI___minds14/en-US/1.0.0/aa40414f15e0f919231d617440192034af844835dc1e6a697f4b552e0551fd26. Subsequent calls will reuse this data.


In [11]:
# minds = minds.remove_columns(["english_transcription", "intent_class", "lang_id"])

In [12]:
minds

Dataset({
    features: ['path', 'audio', 'transcription'],
    num_rows: 100
})

In [13]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [14]:
show_random_elements(minds.remove_columns(["path", "audio"]))

,transcription
0,can I set up a joint account
1,is like you open a joint account tell me about processes
2,I would like to know what my current account balance shows
3,how do I set up a joint account
4,yes I'd like to set up a joint account I'm allowed to anyone
5,I'd like to set up a joint account
6,hi I was trying to set up a joint account is there a somewhere on the app that I can do that or do I have to do that through you
7,what is my account balance
8,show me my current countdown
9,hello I'm calling about my account balance


In [15]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower()
    return batch

minds = minds.map(remove_special_characters)

  0%|          | 0/100 [00:00<?, ?ex/s]

In [16]:
vocab_dict = {'vocab': [],'all_text': []}
def extract_all_chars(batch):
  global vocab_dict
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  vocab_dict['vocab'].extend(vocab)
  vocab_dict['all_text'].append(all_text)
  return vocab_dict

m1 = minds.remove_columns(["path", "audio"])
vocabs = m1.map(extract_all_chars)

  0%|          | 0/100 [00:00<?, ?ex/s]

In [17]:
vocab_dict['vocab'] = list(set(vocab_dict['vocab']))

In [18]:
print(vocab_dict)

{'vocab': ['w', 'p', "'", 'y', 'h', 't', 'j', 'o', 'a', 'n', 'g', 'l', 'u', 'd', 'e', 's', 'k', ' ', 'c', 'v', 'i', 'm', 'r', 'f', 'b'], 'all_text': ['i   w o u l d   l i k e   t o   s e t   u p   a   j o i n t   a c c o u n t   w i t h   m y   p a r t n e r', 'h e n r y   c o u n t y   s e t   u p   a   j o i n t   a c c o u n t   w i t h   m y   w i f e   a n d   w h e r e   a r e   t h e y   a t', "h i   i ' d   l i k e   t o   s e t   u p   a   j o i n t   a c c o u n t   w i t h   m y   p a r t n e r   i ' m   n o t   s e e i n g   t h e   o p t i o n   t o   d o   i t   o n   t h e   a p p   s o   i   c a l l e d   i n   t o   g e t   s o m e   h e l p   c a n   i   d o   i t   o v e r   t h e   p h o n e   w i t h   y o u   a n d   g i v e   y o u   t h e   i n f o r m a t i o n", 'h o w   d o   i   s t a r t   a   j o i n t   a c c o u n t', 'c a n   y o u   h e l p   m e   s e t   u p   a   j o i n t   b a n k   a c c o u n t', 'h o w   t o   s e t   u p   a   j o i n t   a c 

In [19]:
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base")

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [20]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

In [21]:
minds = minds.train_test_split(test_size=0.2)

In [22]:
minds['train']['audio'][0]['array']

array([ 8.8242796e-06,  6.1131783e-07, -1.0533935e-05, ...,
       -1.2824577e-05,  1.3413760e-05,  1.1720552e-05], dtype=float32)

In [23]:
class SpeechTransform:
  def __init__(self, p=0.5):
    self.p = p

  def __call__(self, y):
    if np.random.rand() < self.p:
      return self.apply(y)
    return y

  def apply(self, y: np.ndarray):
      raise NotImplementedError

In [24]:
class ClipEffect(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, rate=0.25):
    super().__init__(p)
    self.rate = rate
    self.sr = sr
  
  def apply(self, x):
    clip_chain = augment.EffectChain().clip(self.rate)
    return clip_chain.apply(x, src_info={'rate': self.sr})

In [25]:
class PitchShift(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, max_rate=200):
    super().__init__(p)
    self.sr = sr
    self.max_rate = max_rate

  def apply(self, x):
    rate = np.random.randint(-self.max_rate, self.max_rate)
    return augment.EffectChain().pitch(rate).rate(self.sr).apply(x, src_info={'rate': self.sr})

In [26]:
class ReverbEffect(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, max_rate=101):
    super().__init__(p)
    self.sr = sr
    self.max_rate = max_rate
  
  def apply(self, x):
    rnd_rate = np.random.randint(0, self.max_rate)
    return augment.EffectChain().reverb(50, 50, rnd_rate).channels(1).apply(x, src_info={'rate': self.sr})

In [27]:
class TimeDropout(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, max_seconds=0.5):
    super().__init__(p)
    self.sr = sr
    self.max_seconds = max_seconds
  
  def apply(self, x):
    return augment.EffectChain().time_dropout(max_seconds=self.max_seconds).apply(x, src_info={'rate': self.sr})

In [28]:
class TimeShift(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, min_shift=0.7, max_shift=2):
    super().__init__(p)
    self.sr = sr
    self.max_shift = max_shift
    self.min_shift = min_shift
  
  def apply(self, x):
    x = x.numpy()
    rate = np.random.uniform(self.min_shift, self.max_shift)
    return torch.as_tensor(librosa.effects.time_stretch(x, rate))

In [29]:
class Compose:
  def __init__(self, transforms):
    self.transforms = transforms
  def __call__(self, x):
    for tr in self.transforms:
      y = tr(x)
    return y

In [30]:
aug_transforms = Compose([ClipEffect(p=0.6),
                          PitchShift(p=0.6),
                          ReverbEffect(p=0.6),
                          TimeDropout(p=0.6),
                          TimeShift(p=0.8)])

In [31]:
minds

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription'],
        num_rows: 80
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription'],
        num_rows: 20
    })
})

In [65]:
def apply_transforms(batch):
      audio = batch["audio"] 
      sp_arr = aug_transforms(torch.Tensor(audio['array']))
      batch['audio']['array'] = sp_arr
      return batch

minds = minds.map(apply_transforms)

  0%|          | 0/80 [00:00<?, ?ex/s]

  0%|          | 0/20 [00:00<?, ?ex/s]

In [32]:
vocab_list = vocab_dict['vocab']

vocab_list = {v: k for k, v in enumerate(vocab_list)}
vocab_list

{'w': 0,
 'p': 1,
 "'": 2,
 'y': 3,
 'h': 4,
 't': 5,
 'j': 6,
 'o': 7,
 'a': 8,
 'n': 9,
 'g': 10,
 'l': 11,
 'u': 12,
 'd': 13,
 'e': 14,
 's': 15,
 'k': 16,
 ' ': 17,
 'c': 18,
 'v': 19,
 'i': 20,
 'm': 21,
 'r': 22,
 'f': 23,
 'b': 24}

In [33]:
vocab_list["[UNK]"] = len(vocab_list)
vocab_list["[PAD]"] = len(vocab_list)
print(len(vocab_list))

27


In [34]:
with open('/content/drive/MyDrive/vocab.json', 'w') as vocab_file:
    json.dump(vocab_list, vocab_file)

In [35]:
tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [36]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [37]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [38]:
minds["train"]

Dataset({
    features: ['path', 'audio', 'transcription'],
    num_rows: 80
})

In [39]:
rand_int = random.randint(0, len(minds["train"]))

print("Target text:", minds["train"][rand_int]["transcription"])
print("Input array shape:", np.asarray(minds["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", minds["train"][rand_int]["audio"]["sampling_rate"])

Target text: i would like to set up a joint account with my partner
Input array shape: (173398,)
Sampling rate: 16000


In [40]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [41]:
minds_tr = minds.map(prepare_dataset, remove_columns=minds.column_names["train"], num_proc=4)

#0:   0%|          | 0/20 [00:00<?, ?ex/s]

#1:   0%|          | 0/20 [00:00<?, ?ex/s]

#2:   0%|          | 0/20 [00:00<?, ?ex/s]

#3:   0%|          | 0/20 [00:00<?, ?ex/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__`

#1:   0%|          | 0/5 [00:00<?, ?ex/s]

#0:   0%|          | 0/5 [00:00<?, ?ex/s]

#2:   0%|          | 0/5 [00:00<?, ?ex/s]

#3:   0%|          | 0/5 [00:00<?, ?ex/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__`

In [42]:
def collate_function(batch, processor, padding=True, max_length=None, 
                     max_length_labels=None, pad_to_multiple_of=None, pad_to_multiple_of_labels=None):
  b_X = [{"input_values": sample["input_values"]} for sample in batch]
  b_Y = [{"input_ids": sample["labels"]} for sample in batch]
  features = processor.pad(b_X,
            padding=padding,
            max_length=max_length,
            pad_to_multiple_of=pad_to_multiple_of,
            return_tensors="pt") 
  with processor.as_target_processor():
    batchY = processor.pad(b_Y,
        padding=padding,
        max_length=max_length_labels,
        pad_to_multiple_of=pad_to_multiple_of_labels,
        return_tensors="pt")
  labels = batchY["input_ids"].masked_fill(batchY.attention_mask.ne(1), -100)
  features["labels"] = labels
  return features

In [43]:
collate_fn = partial(collate_function, processor=processor, padding=True, max_length=None, 
                     max_length_labels=None, pad_to_multiple_of=None, pad_to_multiple_of_labels=None)

In [44]:
version = 2
model_save_path = "/content/drive/MyDrive/wav2vec_finetuning/"
model_name = f"wav2vec_{version}"
batch_size =3
epochs = 100
lr = 0.0001

In [45]:
train_dataloader = torch.utils.data.DataLoader(minds_tr["train"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [46]:
batch_sample = next(iter(train_dataloader))

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [47]:
print(batch_sample.keys())

dict_keys(['input_values', 'labels'])


In [48]:
print(batch_sample["input_values"].shape)
print(batch_sample["labels"].shape)

torch.Size([3, 137898])
torch.Size([3, 78])


In [49]:
batch_sample['labels'][0]

tensor([ 4, 14, 11, 11,  7, 25, 20, 25,  6, 12, 15,  5, 25,  0,  8,  9,  5, 14,
        13, 25,  5,  7, 25, 16,  9,  7,  0, 25,  4,  7,  0, 25, 21, 12, 18,  4,
        25, 21,  7,  9, 14,  3, 25, 20, 25, 18, 12, 22, 22, 14,  9,  5, 11,  3,
        25,  4,  8, 19, 14, 25, 20,  9, 25, 21,  3, 25, 24,  8,  9, 16, 25,  8,
        18, 18,  7, 12,  9,  5])

In [50]:
print(len(train_dataloader))

27


In [51]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes = True
)

/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Downloading:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['project_q.bias', 'quantizer.codevectors', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_q.weight', 'project_hid.weight', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [128]:
os.makedirs(model_save_path, exist_ok=True)

In [52]:
model.freeze_feature_extractor()

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1635: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [138]:
def compute_metrics(labels, preds):
    preds = torch.argmax(preds, axis=-1)
    labels[labels == -100] = processor.tokenizer.pad_token_id
    print(' preds' , preds)
    print('labels', labels)
    pred_str = processor.batch_decode(preds)
    label_str = processor.batch_decode(labels, group_tokens=False)
    return wer(label_str, pred_str)

In [139]:
optimizer = AdamW(model.parameters(), lr=lr)

In [140]:
num_training_steps = epochs*len(train_dataloader)
# lr_scheduler = LinearLR(
#     optimizer=optimizer,
#     start_factor=0.5, total_iters=4
# )
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [141]:
scaler = torch.cuda.amp.GradScaler()

In [142]:
model = model.to(device)
model = model.train()

In [143]:
torch.save(model, os.path.join(model_save_path, model_name+".pt"))
processor.save_pretrained(os.path.join(model_save_path, model_name+"_vocab"))

In [144]:
def train_step(train_dataloader, optimizer, lr_scheduler, processor, verbose_at=5, print_sentences=False):
    losses = []
    wers = []
    for step, data in enumerate(train_dataloader):
        optimizer.zero_grad()
        # move inputs to device
        batch = {k: v.to(device) for k, v in data.items()}
        with torch.cuda.amp.autocast():
          outputs = model(**batch)
        loss = outputs.loss
        losses.append(loss.cpu().detach().numpy())
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        lr_scheduler.step()
        preds = outputs.logits
        labels = data["labels"]
        metrics = compute_metrics(labels, preds)
        wers.append(metrics)
        if step%verbose_at == 0:
            print(f"WER: {metrics} \t LOSS: {loss.cpu().detach().numpy()}")
            if print_sentences:
              for l, p in zip(labels, preds):
                  predicted_str = processor.tokenizer.decode(torch.argmax(p, dim =-1))
                  label_str = processor.tokenizer.decode(l)
                  print(f"True: {label_str}; Predicted: {predicted_str}")
    return {"loss": np.mean(losses), "wer":np.mean(wers)}

In [145]:
no_epochs = range(epochs)
epochs_progress_bar = tqdm.tqdm(no_epochs)
curr_best_loss = 1e10
for n in range(1):
    res = train_step(train_dataloader, optimizer, lr_scheduler, 
                     processor, verbose_at=len(train_dataloader)//2)
    res["best_loss"] = curr_best_loss
    if curr_best_loss > res["loss"]:
        print("Best model, saving at ",model_save_path)
        torch.save(model, os.path.join(model_save_path, model_name+".pt"))
        curr_best_loss = res["loss"]
        res["best_loss"] = curr_best_loss
    epochs_progress_bar.update(1)

  1%|          | 1/100 [00:57<1:34:12, 57.10s/it]
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


 preds tensor([[26, 26, 26,  ..., 26, 26, 26],
        [26, 26, 26,  ..., 26, 26, 26],
        [26, 26, 26,  ..., 26, 26, 26]], device='cuda:0')
labels tensor([[21, 15, 25, 15, 11, 17, 25, 19, 15, 18,  7, 25, 14, 23, 25,  9,  7, 14,
         25, 12, 20, 25, 24, 25, 10, 23, 15,  4, 14, 25, 24,  2,  2, 23, 12,  4,
         14, 25, 13, 15, 14, 21, 25, 22,  0, 25, 20, 24,  6, 14,  4,  7,  6, 25,
         15, 11, 22, 25,  4, 23, 14, 25,  9,  7,  7, 15,  4,  1, 25, 14, 21,  7,
         25, 23, 20, 14, 15, 23,  4, 25, 14, 23, 25, 17, 23, 25, 15, 14, 25, 23,
          4, 25, 14, 21,  7, 25, 24, 20, 20, 25,  9, 23, 25, 15, 25,  2, 24, 19,
         19,  7, 17, 25, 15,  4, 25, 14, 23, 25,  1,  7, 14, 25,  9, 23, 22,  7,
         25, 21,  7, 19, 20, 25,  2, 24,  4, 25, 15, 25, 17, 23, 25, 15, 14, 25,
         23,  5,  7,  6, 25, 14, 21,  7, 25, 20, 21, 23,  4,  7, 25, 13, 15, 14,
         21, 25,  0, 23, 12, 25, 24,  4, 17, 25,  1, 15,  5,  7, 25,  0, 23, 12,
         25, 14, 21,  7, 25, 15,  4,  

KeyboardInterrupt: ignored

In [89]:
model_path = os.path.join(model_save_path, model_name+".pt")
pipeline_path= os.path.join(model_save_path, model_name+"_vocab")

In [53]:
model = model.eval()

In [56]:
minds_te = minds.map(apply_transforms, remove_columns=minds.column_names["test"])
minds_te = minds.map(prepare_dataset, remove_columns=minds.column_names["test"], num_proc=4)

In [57]:
print(minds_te)

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 80
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 20
    })
})


In [58]:
test_dataloader = torch.utils.data.DataLoader(minds_te["test"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [59]:
next(iter(test_dataloader))['input_values']

tensor([[-0.0027, -0.0020, -0.0009,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0020,  0.0015,  0.0003,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0028, -0.0114, -0.0154,  ..., -0.0155, -0.0021,  0.0058]])

In [70]:
for step, data in enumerate(train_dataloader):
            batch = {k: v for k, v in data.items()}
            with torch.cuda.amp.autocast():
              outputs = model(**batch)
              logits = outputs.logits
              predicted_ids = torch.argmax(logits, dim =-1)
              transcriptions = " ".join(processor.tokenizer.convert_ids_to_tokens(predicted_ids[0].tolist()))
              # transcriptions = tokenizer.decode(predicted_ids[0])
              print(transcriptions)
              print(tokenizer.decode(data['labels'][0]))

s k a k a a a a k a a a a a a a a a a a a s s g e a s s s e a g e a a a g a a g a g a c g c k c c s c s f o s o e s e s s s s h h h e o e s s e s e u r e h o u o u a u u a a c c c c m s e e m m s e e v s s d s s s s k k o s s e s s s s s s s b s s s s s s s f f ' ' f f f f f f a a s h s v s s e k ' e e e e e e e e e ' h h s o b h h f f e d s s s s s s s s s e e m m m e e f u u s s s k s s h d d s s s s e e e e e b b b b b s c s s y e e e s s s e f a a     c j j j d e e e e e e e e e u j j j d c c c d s s e e e s s s s s s k k r c j j f r r s r a a a a a a g a g a a g a s g s g a s a s s s s s s s [PAD] s g s s s s g s s a g a s e a s s s e s e a s s a e s s s a e s s s a g s s s a s a s s s s a s e s s [PAD] s s s s [PAD] s g a g a g g s s g s g a g c k k k k k k k k c k   k c c c c k d e e k [UNK] s o o   k c k k s s s s s k k k c c c g a g g g a a g a g g k g c e e g g a a a a k a s g s s a s s s s s s g s s s s a s s s s s s e s e s s s a s s s s a s a s g s s a s e a s a s e a s s 

KeyboardInterrupt: ignored